In [1]:
import torch
import cv2
import numpy as np
from PIL import Image
import imageio
import sys
import os
sys.path.append("""/home/jonfrey/ASL/src""")
os.chdir("/home/jonfrey/ASL")
from pseudo_label import PseudoLabelGenerator
from visu import Visualizer

scannet = '/home/jonfrey/datasets/scannet/'

In [2]:
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=False, num_classes=41)

In [23]:
import pandas as pd
from pytorch_lightning import metrics as pl_metrics

class PseudoLabelEvaluator():
    def __init__(self, settings):
        """
        settings: list of dicts with "name", "cfg" params for PLG
        """
        self.settings = settings
        self.df = pd.DataFrame( columns=['name', 'idx', 'acc_raw', 'acc_pseudo'] )
        self.visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=False, num_classes=41)
        self.acc = pl_metrics.classification.Accuracy()
    
    def evaluate_all_settings(self):
        for j, s in enumerate( self.settings):
            print(f"Evaluate Setting {j}/{len(self.settings)-1}: \n", s)
            plg = PseudoLabelGenerator(visu=self.visu, visu_active=False, **s['cfg'])
            
            for i in range(0,10):
            
                dept, label_pseudo, label_pred_raw = plg.calculate_label(i)
                label_gt = plg.get_gt_label(i)
                m  =  label_gt > -1
                acc_pseudo = float( self.acc(torch.from_numpy( label_pseudo[m]), torch.from_numpy( label_gt[m])) )
                acc_raw = float( self.acc(torch.from_numpy( label_pred_raw[m]), torch.from_numpy( label_gt[m])) )
                self.df = self.df.append( {'name': s['name'], 'idx': i, 'acc_raw': acc_raw, 'acc_pseudo': acc_pseudo  },
                          ignore_index=True)
    def print_results(self):
        return self.df

    def evaluate_df(self):
        return self.df.groupby('name').mean()

settings = [
# {
# 'name': "Super",
# 'cfg': {
#     "refine_superpixel": True,
#     "base_path": "/home/jonfrey/results/scannet_pseudo_label",
#     "cfg_loader": {"ignore_depth":True}
# }
# },
# {
# 'name': "Not Super",
# 'cfg': {
#     "refine_superpixel": False,
#     "base_path": "/home/jonfrey/results/scannet_pseudo_label",
#     "cfg_loader": {"ignore_depth":True}
#     }
# },
{
'name': "Not Super",
'cfg': {
    "refine_superpixel": False,
    "window_size": 3,
    "base_path": "/home/jonfrey/results/scannet_pseudo_label",
    "cfg_loader": {"ignore_depth":True}
    }
}
]    
ple = PseudoLabelEvaluator(settings)
ple.evaluate_all_settings()
ple.evaluate_df()


# index = 0
# dept, label_pseudo, label_pred_raw = plg.calculate_label(index)
# label_gt = plg.get_gt_label(index)
# img = plg.get_img(index)

# visu.plot_segmentation(label_pred_raw, jupyter=True)
# visu.plot_segmentation(label_pseudo, jupyter=True)
# visu.plot_segmentation(label_gt, jupyter=True)


Evaluate Setting 0/0: 
 {'name': 'Not Super', 'cfg': {'refine_superpixel': False, 'window_size': 3, 'base_path': '/home/jonfrey/results/scannet_pseudo_label', 'cfg_loader': {'ignore_depth': True}}}


,acc_raw,acc_pseudo
name,,
Not Super,0.822684,0.815538


In [21]:
cfg ={
    "refine_superpixel": False,
    "window_size": 5,
    "base_path": "/home/jonfrey/results/scannet_pseudo_label",
    "cfg_loader": {"ignore_depth":True}
}
plg = PseudoLabelGenerator(visu=visu, visu_active=False, **cfg)
idx = 20
depth, label_pseudo , label_pred_raw = plg.calculate_label(idx)
label_gt = plg.get_gt_label(idx)
# plg._visu_seg_forwarded(seg)

# visu.plot_segmentation(seg= label_gt + 1, jupyter=True)  
# visu.plot_segmentation(seg= label_pred_raw + 1, jupyter=True)  

acc = pl_metrics.classification.Accuracy()
m = label_gt != -1
acc_pseudo = float( acc(torch.from_numpy( label_pseudo[m]), torch.from_numpy( label_gt[m])) )
acc_raw = float( acc(torch.from_numpy( label_pred_raw[m]), torch.from_numpy( label_gt[m])) )
acc_pseudo, acc_raw

(0.781923770904541, 0.7605735063552856)

(0.7581599950790405, 0.757673442363739)

In [5]:
seg

Array([[ 0,  0,  0, ...,  2,  2,  2],
       [ 0,  0,  0, ...,  2,  2,  2],
       [ 0,  0,  0, ...,  2,  2,  2],
       ...,
       [ 0,  0,  0, ..., 39, 39, 39],
       [ 0,  0,  0, ..., 39, 39, 39],
       [ 0,  0,  0, ..., 39, 39, 39]], dtype=int32)